In [29]:
import pickle
import fairness_metrics
import pandas as pd
from tqdm import tqdm
import utils

In [30]:
# read the data
ideology_df = pd.read_csv('./data/processed_annotated_comments.csv')
ideology_df['label'] = ideology_df['label'].apply(lambda x: None if x not in ['left', 'right'] else x)
ideology_df.dropna(inplace=True)
gender_df = pd.read_csv('./data/jigsaw/gender.csv')
race_df = pd.read_csv('./data/jigsaw/race.csv')
disability_df = pd.read_csv('./data/jigsaw/disability.csv')
sexual_orientation_df = pd.read_csv('./data/jigsaw/sexual_orientation.csv')
dfs = [ideology_df, gender_df, race_df, disability_df, sexual_orientation_df]
names = ['ideology', 'gender', 'race', 'disability', 'sexual orientation']

In [31]:
# load gold standard moderation if raw results are saved

# with open('./results/<openai_moderation>.pkl', 'rb') as handle: # comments_moderated_openai
#     gold1 = pickle.load(handle)

# with open('./results/<perspective_moderation>.pkl', 'rb') as handle: # comments_moderated_perspective_nonmanager
#     gold2 = pickle.load(handle)

# with open('./results/<google_palm_moderation>.pkl', 'rb') as handle: # comments_moderation_google
#     gold3 = pickle.load(handle)

# with open('./results/<clarifai_moderation>.pkl', 'rb') as handle: # comment_moderation_clarifai
#     gold4 = pickle.load(handle)

# gold1 = utils.process_openai(gold1)
# gold2 = utils.process_perspective(gold2)
# gold3 = utils.process_google(gold3)
# gold4 = utils.process_clarifai(gold4)

In [32]:
# load preprocessed results
with open('./results/moderation_results.pkl', 'rb') as file:
    fairness_results = pickle.load(file)


gold1 = {k:v['openai'] for k,v in fairness_results.items()}
gold2 = {k:v['perspective'] for k,v in fairness_results.items()}
gold3 = {k:v['google'] for k,v in fairness_results.items() if v['google'] in [True, False]}
gold4 = {k:v['clarifai'] for k,v in fairness_results.items()}

gold1 = {k: 1 if v is True else 0 for k,v in gold1.items()}
gold2 = {k: 1 if v is True else 0 for k,v in gold2.items()}
gold3 = {k:1 if v is True else 0 for k,v in gold3.items()}
gold4 = {k:1 if v is True else 0 for k,v in gold4.items()}

In [ ]:
golds = [gold1, gold2, gold3, gold4]

methods = ['openai', 'perspective','google', 'clarifai']
perturbations = ['german', 'gpt_3.5_turbo']

results = []
global_results = {}
for method in methods:
    global_results[method] = {}

perturbation_map = {}


with open('./results/comments_backtranslated_german_similarity.pkl', 'rb') as handle:
        perturbation_map[perturbations[0]] = pickle.load(handle)

with open('./results/comment_paraphrased_gpt-3.5_final.pkl', 'rb') as handle:
        perturbation_map[perturbations[1]] = pickle.load(handle)

for perturbation in perturbations:
    with open(f'./results/moderation_results_fairness_perturbed_{perturbation}.pkl', 'rb') as handle:
        fairness_results = pickle.load(handle)
    
    phrase_map = perturbation_map[perturbation]
    if perturbation == "german":
        phrase_map = {k:v['augmented'] for k,v in phrase_map.items() if v['score'] > 0.85 and v['score'] != 1.0}

    for gold, method in zip(golds, methods):
        local_gold = gold.copy()
        
        # check to ignore NULL values for phrases where moderation did not run
        fairness_results = {k:v for k,v in fairness_results.items() if fairness_results[k][method] in [True, False]}
        
        data = {k:int(fairness_results[v][method]) for k,v in phrase_map.items() if v in fairness_results.keys()}

        local_gold = {k:v for k,v in local_gold.items() if k in data.keys() and v in [True, False]}

        # create lists and compute robustness
        global_results[method][perturbation] = {}
        for subset, name in zip(dfs, names):
            df = subset.copy()
            df = df[df['text'].isin(list(local_gold.keys()))]
            a = [local_gold[k] for k in df['text'].tolist()]
            b = [data[k] for k in df['text'].tolist()]
            
            r = fairness_metrics.robustness(a, b)
            global_results[method][perturbation][name] = r*100
            print(r*100, method, perturbation, name)


In [ ]:
global_results